In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_data = pd.read_csv("data/train_data.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")
train_data = train_data.fillna(method = 'bfill')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3859200 entries, 0 to 3859199
Data columns (total 13 columns):
 #   Column   Dtype  
---  ------   -----  
 0   TurbID   int64  
 1   Day      int64  
 2   Tmstamp  object 
 3   Wspd     float64
 4   Wdir     float64
 5   Etmp     float64
 6   Itmp     float64
 7   Ndir     float64
 8   Pab1     float64
 9   Pab2     float64
 10  Pab3     float64
 11  Prtv     float64
 12  Patv     float64
dtypes: float64(10), int64(2), object(1)
memory usage: 382.8+ MB


In [ ]:
def make_train_data(data):
    train_x, train_y = [], []
    for i in tqdm(sorted(pd.unique(data['TurbID']))):
        tmp_data = data[data['TurbID'] == i]
        for j in range(1, 201 - 6):  # 6일
            day_list       = [x for x in range(j, j+5)]
            label_day_list = [y for y in range(j+5, j+7)]
            
            train_tmp = tmp_data[tmp_data['Day'].isin(day_list)]
            label_tmp = tmp_data[tmp_data['Day'].isin(label_day_list)]['Patv']
            
            train_tmp = train_tmp.drop(["TurbID", "Day"], axis=1)
            train_x.append(np.array(train_tmp))
            train_y.append(np.array(label_tmp))
    return train_x, train_y

In [ ]:
tms_list = list(pd.unique(train_data['Tmstamp']))
train_data['Tmstamp'] = train_data['Tmstamp'].apply(lambda x: tms_list.index(x)+1)
train_x, train_y = make_train_data(train_data)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
train_x = np.array(train_x).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
train_y = np.array(train_y)

train_x.shape, train_y.shape

In [8]:
model.fit(train_x, train_y, epochs=10, batch_size=128);

2022-07-09 13:15:40.859916: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 823553280 exceeds 10% of free system memory.
2022-07-09 13:15:41.236905: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 823553280 exceeds 10% of free system memory.


Epoch 1/10


2022-07-09 13:15:42.999226: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  3/204 [..............................] - ETA: 7s - loss: 312189.1562 - mae: 357.0135  

2022-07-09 13:15:43.537265: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


204/204 [==============================] - 10s 37ms/step - loss: 187974.3125 - mae: 330.6494
Epoch 2/10
204/204 [==============================] - 8s 37ms/step - loss: 162182.8906 - mae: 314.8804
Epoch 3/10
204/204 [==============================] - 8s 37ms/step - loss: 152358.3125 - mae: 302.9175
Epoch 4/10
204/204 [==============================] - 8s 38ms/step - loss: 144841.1250 - mae: 292.7060
Epoch 5/10
204/204 [==============================] - 8s 37ms/step - loss: 136701.9688 - mae: 282.8264
Epoch 6/10
204/204 [==============================] - 8s 37ms/step - loss: 130888.4609 - mae: 274.9980
Epoch 7/10
204/204 [==============================] - 8s 37ms/step - loss: 125617.6016 - mae: 268.5308
Epoch 8/10
204/204 [==============================] - 8s 37ms/step - loss: 120703.5156 - mae: 262.2213
Epoch 9/10
204/204 [==============================] - 8s 37ms/step - loss: 116745.2500 - mae: 256.8172
Epoch 10/10
204/204 [==============================] - 8s 37ms/step - loss: 112719.

In [9]:
test_data_list = [x for x in range(196, 201)]
test_data      = train_data[train_data["Day"].isin(test_data_list)]
test_data      = test_data.drop(["TurbID", "Day"], axis=1)
test_data      = np.array(test_data).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
test_data.shape

(134, 720, 11)

In [10]:
sample_submission['Patv'] = model.predict(test_data).reshape(-1)
sample_submission.to_csv("output/baseline1.csv", index=False)

5/5 [==============================] - 0s 10ms/step
